In [2]:
#This is a program that reads through all of the section headings in Chapter 1 of Schaller's Polymer Chemistry
#It creates 4 lists: 1) chapter titles 2) chapter summaries 3) uncleaned chapters and 4) url endings
#After reading the data from the chapter page, it uses the url endings to get the webpage of each of the chapters and read the 
#text from each chapter, storing it in the list, uncleaned_sub_sects
#An attempt is made to start cleaning the data, but it still needs some work

from IPython.core.display import clear_output
import time
from time import sleep
from random import randint
from warnings import warn
import pandas as pd
from bs4 import BeautifulSoup
from requests import get

#making sure english
headers = {"Accept-Language": "en-US, en;q=0.5"}

#initializing time/requests for monitoring
start_time = time.time()
requests = 0

#url for chapter page, libre_response is the html
url = 'https://chem.libretexts.org/Bookshelves/Organic_Chemistry/Book%3A_Polymer_Chemistry_(Schaller)/1%3A_Monomers_and_Polymers'
libre_response = get(url)

#/Book%3A_Polymer_Chemistry_(Schaller)/1%3A_Monomers_and_Polymers/1.1%3A_Difunctional_Carboxyloids
#want it to go thru all of the headings on that page and read them into separate something, 
#in html data-page-id is unique marker for each "box"

#creates a BeautifulSoup object of the chapter 1 page
html_chap1 = BeautifulSoup(libre_response.text, 'html.parser')
#print(html_chap1)

#creating two lists, for chapter titles and url titles
chapter_titles = []
url_titles = []

#using beautifulSoup to navigate through the html and find the boxes for each chapter/summary/title
unit_boxes = html_chap1.find_all('dl', class_ = 'mt-listing-detailed')
summaries = []

print(type(unit_boxes))
print(len(unit_boxes))
print('')
print(unit_boxes[0])
print('')


#selects the first box from the unit boxes and prints out the title and the summary paragraph 
first_box = unit_boxes[0] 
first_box_title = first_box.a.text
print(first_box_title)
print('')
summary_text = first_box.dd.text
print(summary_text)
print('')

#next step is to have a for-loop go through all of the unit boxes and select the title
#to get url addition, want to 1) delete colon, 2) replace first space with %3A_ and second space with _
# also need to figure out if/else to get rid of solutions chapter
# 3) put titles into dataframe, 4) use url_titles to get to each webpage and read all the shit

for unit_box in unit_boxes:
    unit_box_title = str(unit_box.a.text)
    sect_url = unit_box_title.replace(': ', '%3A_')
    #print(url)
    final_url = sect_url.replace(' ', '_')
    #print(final_url)
    summaries.append(unit_box.dd.text)
    url_titles.append(final_url)
    chapter_titles.append(unit_box_title)
    
print(chapter_titles)
print('')
print(url_titles)


<class 'bs4.element.ResultSet'>
9

<dl class="mt-listing-detailed"><dt class="mt-listing-detailed-title"><a class="internal" href="https://chem.libretexts.org/Bookshelves/Organic_Chemistry/Book%3A_Polymer_Chemistry_(Schaller)/1%3A_Monomers_and_Polymers/1.1%3A_Difunctional_Carboxyloids" rel="internal" title="1.1: Difunctional Carboxyloids"><span class="mt-icon-article-topic"></span>1.1: Difunctional Carboxyloids</a></dt><dd class="mt-listing-detailed-overview">The first synthetic polymer to really seize the public's consciousness -- the first one to change the course of world events -- was nylon.  Not long after its discovery in the 1930s, it had supplanted silk as the material of choice for stockings, having transformed a luxury item into a widely available commodity.  Nylon came just in time for the Second World War.  Soon, parachutes, too, went from being made of silk to being fabricated from nylon.</dd></dl>

1.1: Difunctional Carboxyloids

The first synthetic polymer to really seiz

In [3]:
#New loop, starting from the Schaller Polymer Chemistry textbook, to request from LibreTexts each section(1.1, 1.2, etc..)
#After looping through, the chapters are assigned to the titles and put into a dataframe
#Potentially a monitoring method


headers = {"Accept-Language": "en-US, en;q=0.5"}
uncleaned_sub_sects = []

#initializing time/requests for monitoring
start_time = time.time()
requests = 0

updated_url = 'https://chem.libretexts.org/Bookshelves/Organic_Chemistry/Book%3A_Polymer_Chemistry_(Schaller)/1%3A_Monomers_and_Polymers/'

for url_title in url_titles:
    
    sect_response = get(updated_url + url_title, headers = headers)

    #pausing loop for random time of 8-15 seconds
    sleep(randint(8,15))
        
    #monitoring requests
    requests += 1
    elapsed_time = time.time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)
        
        #throwing warnings for non-200 status code
    #if response.status_code !=200:
     #   warn('Reqest: {}; Status code: {}'.format(requests, response.status_code))

        
    html_sect = BeautifulSoup(sect_response.text, 'html.parser')
    sect_text = html_sect.find('section', class_ = 'mt-content-container')
    uncleaned_sub_sects.append(sect_text.text)

Request:9; Frequency: 0.07897991176700662 requests/s


In [4]:
#cleans each subsection text of the html at the end
for i in range(0, len(uncleaned_sub_sects)):
    uncleaned_sub_sects[i] = uncleaned_sub_sects[i][0:uncleaned_sub_sects[i].index("/*<!")]
    #print(uncleaned_sub_sects[3])
    



In [7]:
#Getting cleaned chapters/chapter titles together in a dictionary, then exporting to json

import json 

print("uncleaned_sub_sects type: " , type(uncleaned_sub_sects))

#creates a new dictionary with chapter titles being keys and chapters being there values
cleaned_dict = dict(zip(chapter_titles, uncleaned_sub_sects))

print("cleaned dictionary type: ", type(cleaned_dict))

with open("chapter_1_Schaller", "w") as outfile:
    json.dump(cleaned_dict, outfile)

uncleaned_sub_sects type:  <class 'list'>
cleaned dictionary type:  <class 'dict'>


In [10]:
#print('')
#print("cleaned dict, chapter 3: ", cleaned_dict["1.8: Polymer Topology"])


cleaned dict, chapter 3:  
Polymers are very large molecules made from smaller ones.  How those smaller units are arranged within the polymer is an issue we haven't addressed very closely yet.  Topology is the study of three-dimensional shapes and relationships, or of how individual parts are arranged within a whole.  Let's take a look at the topology of polymers.
At the simplest level, we have been thinking of polymers as chains of monomers strung together like beads on a string.  We even use the verb "enchain" to describe the act of taking a monomer and tying it into a larger polymer.  The chain is the most basic of polymer structures.  This topology is often referred to simply as a chain or, to underscore the structure, a linear polymer.

 
Even a simple chain, in this context, can have additional structural features worth considering.  These features are connected to how a polymer chain grows up from individual monomers.  If the polymer results from a chain reaction, typically use